In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
    

In [2]:

# URL of the webpage with the table
url = "https://natural-resources.canada.ca/our-natural-resources/energy-sources-distribution/fossil-fuels/crude-oil/oil-pricing/18087#cop"

# Send a GET request to the webpage
response = requests.get(url)


In [3]:
response


<Response [200]>

In [4]:
# Create BeautifulSoup object and specify the parser

soup = BeautifulSoup(response.content, "html.parser")

In [6]:

# to find the dates witin the links: these are the 2 strings that border the date string
sub_link="/crude-oil/oil-pricing/selected-crude-oil-prices-"
sub_link2="-canadian-dollars-cubic-metre"

In [7]:
#creating list that has all the links
all_links=[]

for link in soup.find_all('a'):
    href=link.get('href')
    if sub_link in href:
      all_links.append(href)
    

In [14]:
#creating list that has all the dates
dates=[]
for i in range(len(all_links)):
    x=0
    y=0
    x=all_links[i].find(sub_link)
    y=all_links[i].find(sub_link2)
    x=x+len(sub_link)
    
    new_link=all_links[i][x:y]
    dates.append(new_link)
    
    

# better methods potentially:
    ##use enumerate instead
    ## use split  string.split
    ## regex

In [15]:
# sorting the input list by formatting each date using the strptime() function
try:
    dates.sort(key=lambda date: datetime.strptime(date, "%B-%Y"))
    
except:
    pass



In [16]:
#getting 2 months worth of data just in case
latest_date=dates[0]
second_latest_date=dates[1]
target_link=''
last_month_link=''

In [17]:
for i  in range(len(all_links)):  
    if latest_date in all_links[i]:
        target_link=all_links[i]
        #last_month_link=all_links[i+1]
    if second_latest_date in all_links[i]:
        last_month_link=all_links[i]
    if target_link and  last_month_link:
        break
  # once target link and last month link is found, for loop breaks      
# use regex to be more specific find dates within the 2 strings only

In [25]:
# Send a GET request to the latest data
response1 = requests.get(target_link)

In [26]:
# Send a GET request to the latest-1 data
response2 = requests.get(last_month_link)

In [27]:
# Create BeautifulSoup objects and specify the parser

soup1 = BeautifulSoup(response1.content, "html.parser")
# Find the table element based on its HTML class (e.g., <table class="data-table">)
table1 = soup1.find("table", class_="table-bordered")

soup2 = BeautifulSoup(response2.content, "html.parser")
# Find the table element based on its HTML class (e.g., <table class="data-table">)
table2 = soup2.find("table", class_="table-bordered")

In [32]:


# Check if the table exists
if table1 and table2:
    # Extract table headers (assuming they are in <th> tags)
    headers1 = [header.text.strip() for header in table1.find_all("th")]
    headers2 = [header.text.strip() for header in table2.find_all("th")]

    # Extract table rows (assuming they are in <tr> tags)
    rows1 = []
    rows2 = []
    for row in table1.find_all("tr"):
        # Extract the data cells for each row (assuming they are in <td> tags)
        cells = [cell.text.strip() for cell in row.find_all("td")]
        rows1.append(cells)
        
  #same for second table:      
      
    for row in table2.find_all("tr"):
        # Extract the data cells for each row (assuming they are in <td> tags)
        cells = [cell.text.strip() for cell in row.find_all("td")]
        rows2.append(cells)

 

    # Create a pandas DataFrames
    df1 = pd.DataFrame(rows1, columns=headers1)
    df2= pd.DataFrame(rows2, columns=headers2)

else:
    print("Tables not found on the webpage.")


In [36]:
df1.columns = df1.columns.str.replace('\n', '')
df2.columns = df2.columns.str.replace('\n', '')
